In [1]:
from bidict import bidict
from collections import defaultdict
import itertools
import os
import pandas as pd
import pickle
import lzma

from conllu import parse, parse_incr
import numpy as np
from cp_orth import orth_als
import sktensor

from decomp_pmi import VerbTensor

import logging
logging.basicConfig(level=logging.DEBUG, format='%(levelname)-8s [%(lineno)d] %(message)s')

# DepCC: salience

In [2]:
svo_count = pd.read_csv('/mnt/store/home/makrai/project/verb-tensor/dataframe/depCC/freq0.tsv', sep='\t')

In [3]:
svo_count.head()#sort_values('freq', ascending=False).head()

,nsubj,ROOT,dobj,freq
0,we,support,browser,141767
1,I,have,idea,124640
2,we,reserve,right,105981
3,I,love,it,95660
4,I,have,problem,82073


In [4]:
svo_count, log_total = VerbTensor().append_pmi(svo_count=svo_count)

INFO     [43] Computing marginals..
INFO     [45] Computing 2-marginals..
INFO     [52] Computing Dice..
INFO     [69] Computing PMI variants..
INFO     [89] Computing salience..
INFO     [92] Saving to /mnt/store/home/makrai/project/verb-tensor/dataframe/depCC/assoc0.{}..


In [7]:
svo_count[svo_count.columns[10:]].describe(percentiles=[])

,dice,freq2,pmi,iact_info,salience,iact_sali
count,1.447557e+07,1.447618e+07,1.447618e+07,1.447618e+07,1.447618e+07,1.447618e+07
mean,1.442307e-03,-2.510492e+01,5.106900e+01,7.008352e+01,-1.282083e+03,-1.759441e+03
std,2.979052e-02,1.001235e+00,2.629008e-13,2.827960e-12,5.113209e+01,7.017011e+01
min,1.467822e-07,-2.553450e+01,5.106900e+01,7.008352e+01,-1.304021e+03,-1.789548e+03
50%,5.566710e-06,-2.553450e+01,5.106900e+01,7.008352e+01,-1.304021e+03,-1.789548e+03
max,1.000000e+00,-8.421338e+00,5.106900e+01,7.008352e+01,-4.300693e+02,-5.901970e+02


In [16]:
svo_count[svo_count.freq>1].sort_values('pmi', ascending=False).head()

,nsubj,ROOT,dobj,freq,freq_nsubj,freq_ROOT,freq_dobj,"freq_('nsubj', 'ROOT')","freq_('nsubj', 'dobj')","freq_('ROOT', 'dobj')",dice,freq2,pmi,iact_info,salience,iact_sali
0,we,support,browser,141767,-3.118223,-7.568761,-8.389523,-8.282676,-8.420056,-8.420300,7.090559e-02,-8.421338,51.069,70.083524,-430.069305,-590.197038
2228231,Brandy,want,wedding,2,-17.058767,-6.988374,-12.726340,-24.534500,-24.534500,-17.075068,1.537279e-05,-24.534500,51.069,70.083524,-1252.952389,-1719.464237
2228221,you,know,acting,2,-4.946526,-6.427485,-16.890644,-8.667197,-23.212572,-23.212572,2.802157e-06,-24.534500,51.069,70.083524,-1252.952389,-1719.464237
2228222,I,think,McAvoy,2,-1.945886,-8.159350,-20.949538,-8.402422,-22.534500,-24.534500,4.693065e-07,-24.534500,51.069,70.083524,-1252.952389,-1719.464237
2228223,I,dislike,column,2,-1.945886,-12.400074,-12.252860,-12.977036,-13.935982,-24.534500,4.749184e-07,-24.534500,51.069,70.083524,-1252.952389,-1719.464237


In [17]:
svo_count[svo_count.freq>1].sort_values('iact_info', ascending=False).head()

,nsubj,ROOT,dobj,freq,freq_nsubj,freq_ROOT,freq_dobj,"freq_('nsubj', 'ROOT')","freq_('nsubj', 'dobj')","freq_('ROOT', 'dobj')",dice,freq2,pmi,iact_info,salience,iact_sali
0,we,support,browser,141767,-3.118223,-7.568761,-8.389523,-8.282676,-8.420056,-8.420300,7.090559e-02,-8.421338,51.069,70.083524,-430.069305,-590.197038
2228231,Brandy,want,wedding,2,-17.058767,-6.988374,-12.726340,-24.534500,-24.534500,-17.075068,1.537279e-05,-24.534500,51.069,70.083524,-1252.952389,-1719.464237
2228221,you,know,acting,2,-4.946526,-6.427485,-16.890644,-8.667197,-23.212572,-23.212572,2.802157e-06,-24.534500,51.069,70.083524,-1252.952389,-1719.464237
2228222,I,think,McAvoy,2,-1.945886,-8.159350,-20.949538,-8.402422,-22.534500,-24.534500,4.693065e-07,-24.534500,51.069,70.083524,-1252.952389,-1719.464237
2228223,I,dislike,column,2,-1.945886,-12.400074,-12.252860,-12.977036,-13.935982,-24.534500,4.749184e-07,-24.534500,51.069,70.083524,-1252.952389,-1719.464237


In [10]:
svo_count[svo_count.freq>10].sort_values('dice', ascending=False).head()

,nsubj,ROOT,dobj,freq,freq_nsubj,freq_ROOT,freq_dobj,"freq_('nsubj', 'ROOT')","freq_('nsubj', 'dobj')","freq_('ROOT', 'dobj')",dice,freq2,pmi,iact_info,salience,iact_sali
120910,FarmFlyingUK,feelthere,Simulatio,32,-20.534500,-20.534500,-20.534500,-20.534500,-20.534500,-20.534500,1.0,-20.534500,51.069,70.083524,-1048.676389,-1439.130139
33722,Rovartani,lapok,QL461,113,-18.714321,-18.714321,-18.714321,-18.714321,-18.714321,-18.714321,1.0,-18.714321,51.069,70.083524,-955.721669,-1311.565582
247106,I­­­­­­­­­,began­­­­­­­­­,ago­­­­­­­­­,16,-21.534500,-21.534500,-21.534500,-21.534500,-21.534500,-21.534500,1.0,-21.534500,51.069,70.083524,-1099.745389,-1509.213663
29834,по,прошествии,eshopworld,127,-18.545815,-18.545815,-18.545815,-18.545815,-18.545815,-18.545815,1.0,-18.545815,51.069,70.083524,-947.116250,-1299.756107
266470,Arvioitu,saatavuuspäivä,muuttua,14,-21.727145,-21.727145,-21.727145,-21.727145,-21.727145,-21.727145,1.0,-21.727145,51.069,70.083524,-1109.583580,-1522.714909


In [11]:
svo_count[svo_count.freq>0].sort_values('salience', ascending=False).head()

,nsubj,ROOT,dobj,freq,freq_nsubj,freq_ROOT,freq_dobj,"freq_('nsubj', 'ROOT')","freq_('nsubj', 'dobj')","freq_('ROOT', 'dobj')",dice,freq2,pmi,iact_info,salience,iact_sali
0,we,support,browser,141767,-3.118223,-7.568761,-8.389523,-8.282676,-8.420056,-8.420300,0.070906,-8.421338,51.069,70.083524,-430.069305,-590.197038
1,I,have,idea,124640,-1.945886,-2.639605,-7.420799,-4.249024,-8.187406,-8.034872,0.018066,-8.607092,51.069,70.083524,-439.555607,-603.215376
2,we,reserve,right,105981,-3.118223,-8.633374,-7.919783,-8.824555,-8.742790,-8.696872,0.053704,-8.841054,51.069,70.083524,-451.503787,-619.612222
3,I,love,it,95660,-1.945886,-5.165101,-5.188983,-5.559157,-6.192069,-8.538027,0.018755,-8.988872,51.069,70.083524,-459.052701,-629.971825
4,I,have,problem,82073,-1.945886,-2.639605,-7.996652,-4.249024,-8.924528,-8.619893,0.011950,-9.209880,51.069,70.083524,-470.339364,-645.460851


In [12]:
svo_count[svo_count.freq>0].sort_values('iact_sali', ascending=False).head()

,nsubj,ROOT,dobj,freq,freq_nsubj,freq_ROOT,freq_dobj,"freq_('nsubj', 'ROOT')","freq_('nsubj', 'dobj')","freq_('ROOT', 'dobj')",dice,freq2,pmi,iact_info,salience,iact_sali
0,we,support,browser,141767,-3.118223,-7.568761,-8.389523,-8.282676,-8.420056,-8.420300,0.070906,-8.421338,51.069,70.083524,-430.069305,-590.197038
1,I,have,idea,124640,-1.945886,-2.639605,-7.420799,-4.249024,-8.187406,-8.034872,0.018066,-8.607092,51.069,70.083524,-439.555607,-603.215376
2,we,reserve,right,105981,-3.118223,-8.633374,-7.919783,-8.824555,-8.742790,-8.696872,0.053704,-8.841054,51.069,70.083524,-451.503787,-619.612222
3,I,love,it,95660,-1.945886,-5.165101,-5.188983,-5.559157,-6.192069,-8.538027,0.018755,-8.988872,51.069,70.083524,-459.052701,-629.971825
4,I,have,problem,82073,-1.945886,-2.639605,-7.996652,-4.249024,-8.924528,-8.619893,0.011950,-9.209880,51.069,70.083524,-470.339364,-645.460851


# Get UMBC

In [ ]:
def get_umbc_dict():
    umbc_dir = '/mnt/store/home/makrai/data/language/english/corp/umbc_WebBase/English/'
    freq = defaultdict(int)
    for filen in os.listdir(umbc_dir):
        logging.info(filen)
        for i, sentence in  enumerate(parse_incr(lzma.open(os.path.join(umbc_dir, filen), mode='rt',
                                                           encoding="utf-8"))):
            if not i % 100000:
                logging.debug(i)
            root = sentence.to_tree()
            subj, obj = '', ''
            for child in root.children:
                if 'subj' in child.token['deprel']:
                    if subj:
                        #logging.warn('subj: {}'.format((subj, child.token['lemma'], sentence)))
                        continue
                    subj = child.token['lemma']
                elif child.token['deprel'] == 'obj':
                    if obj:
                        #logging.warn('obj: {}'.format((obj, child.token['lemma'], sentence)))
                        continue            
                    obj = child.token['lemma']
            #if bool(obj) and bool(subj):
            freq[(subj, root.token['lemma'], obj)] += 1
        #pickle.dump(freq, open('/mnt/store/home/makrai/project/verb-tensor/umbc_freq.pkl', mode='wb'))
    return freq

In [ ]:
def get_umbc_df():
    freq = pickle.load(open('/mnt/store/home/makrai/project/verb-tensor/umbc_freq.pkl', mode='rb'))
    freq_df = pd.DataFrame.from_records(list(freq.items()), columns=['svo', 'freq'])
    freq_df[['subj', 'verb', 'obj']] = pd.DataFrame(freq_df.svo.tolist(), index=freq_df.index)                                                                                                                       
    del freq_df['svo']
    return freq_df

# Mazsola DB

In [ ]:
mazsola = pickle.load(open(
    '/mnt/permanent/Language/Hungarian/Dic/sass15-535k-igei-szerkezet/mazsola_adatbazis.pkl',
    mode='rb'))

In [ ]:
mazsola_df =pd.read_csv(
    '/mnt/permanent/Language/Hungarian/Dic/sass15-535k-igei-szerkezet/mazsola_adatbazis_svo_freq.tsv', sep='\t',
    keep_default_na=False)

In [ ]:
mazsola_df, log_total = append_pmi(mazsola_df, compute_freq=False)

In [ ]:
mazsola_df.sort_values('iact_info').head()

# Top triples

In [ ]:
svo_count.sort_values('freq', ascending=False).head()

In [ ]:
svo_count[svo_count.freq>100].sort_values('pmi', ascending=False).head()

In [ ]:
svo_count[(svo_count.freq>100) & (svo_count.ACC != 'NULL')].sort_values('iact_info', ascending=False).head()

In [ ]:
svo_count[(svo_count.ACC != 'NULL')].sort_values('salience', ascending=False).head()

In [ ]:
svo_count.sort_values('iact_sali', ascending=False).head()

## UMBC

In [ ]:
svo_count, log_total = append_pmi(freq_df, modes=['subj', 'verb', 'obj'], compute_freq=False)#, debug_index=2234759)

In [ ]:
svo_count[(svo_count.subj != '') & (svo_count.obj != '')].sort_values('freq', ascending=False).head()

In [ ]:
svo_count[svo_count.freq>100].sort_values('pmi', ascending=False).head()

In [ ]:
svo_count[(svo_count.freq>100) & (svo_count.subj != '') & (svo_count.obj != '')].sort_values(
    'iact_info', ascending=True).head()

In [ ]:
svo_count[(svo_count.subj != '') & (svo_count.obj != '')].sort_values('salience', ascending=False).head()

In [ ]:
svo_count[(svo_count.freq>100)].sort_values('dice', ascending=False).head()